## Importing Libraries

In [9]:
#selenium libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException, ElementClickInterceptedException

import time
import pandas as pd
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

import requests
import urllib.parse

## Setting up a common launch pad for chrome browser with Webdrive

In [2]:
def get_drive_launch(url):
    """
    This function initiates the Chrome browser's driver with the url passed as a parameter and returns the driver instance.
    Parameters:
        url - url of the website
    return:
        driver - driver instance of the chrome browser    
    """
    #creating driver instance
    driver = webdriver.Chrome('./driver/chromedriver.exe')
    driver.maximize_window()
    
    #defining implicit wait
    driver.implicitly_wait(10)
    
    #launching the url
    driver.get(url)
    
    return driver

## 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [3]:
def search_product_amazon(product):
    product = product.lower()
    url = 'https://www.amazon.in/'
    
    drivr = get_drive_launch(url)
    drivr.find_element_by_id("twotabsearchtextbox").send_keys(product)
    drivr.find_element_by_xpath("//input[@type='submit' and @value='Go']").click()
    search_text = drivr.find_element_by_xpath("//div[@class='a-section a-spacing-small a-spacing-top-small']").text
    print("Showing",search_text)
    return drivr

In [4]:
product = 'guitar'
d = search_product_amazon(product)

Showing 1-48 of over 1,000 results for "guitar"


In [5]:
d.quit()

## 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
product = 'guitar'
d = search_product_amazon(product)

Showing 1-48 of over 1,000 results for "guitar"


In [24]:
def fetch_product_details_amazon(product, pages, save_as_csv = False):
    d = search_product_amazon(product)
    num_pages = pages
    page_num = 1
    time.sleep(2)
    main_window = d.current_window_handle
    brands = []
    product_names = []
    ratings = []
    num_ratings = []
    prices = []
    expected_delivery = []
    availabilities = []
    prod_urls = []
    other_details = []
    returns = []

    next_tab = True

    while (page_num < num_pages + 1):
        time.sleep(2)
        page_num += 1


        plp_product_elems = d.find_elements_by_xpath("//div[@data-component-type='s-search-result']")

        for prod_elems in range(len(plp_product_elems)):
            try:
                plp_product_elems[prod_elems].click()
            except StaleElementReferenceException:
                try:
                    plp_product_elems = d.find_elements_by_xpath("//div[@data-component-type='s-search-result']")
                    #d.navigate().refresh()
                    time.sleep(2)
                    plp_product_elems[prod_elems].click()
                except:
                    continue
            except ElementNotInteractableException:
                d.find_element_by_xpath("//a[@class='a-link-normal a-carousel-goto-nextpage s-carousel-button']").click()
                time.sleep(1)
                prod_elems.click()

            all_windows = d.window_handles

            if len(all_windows) > 1:
                d.switch_to.window(all_windows[1])
                next_tab = True
            else:
                next_tab = False

            try:
                try:
                    brands.append(d.find_element_by_id("bylineInfo").text)
                except NoSuchElementException:
                    brands.append("-") 
                try:
                    product_names.append(d.find_element_by_id("productTitle").text.strip())
                except NoSuchElementException:
                    product_names.append("-")
                try:
                    rating = d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@class='a-icon-alt']").text.strip()
                    ratings.append(rating)
                    num_ratings.append(d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@id='acrCustomerReviewText']").text.strip())
                except NoSuchElementException:
                    ratings.append("-")
                    num_ratings.append("-")
                try:
                    prices.append(d.find_element_by_id("priceblock_ourprice").text.strip())
                except NoSuchElementException:
                    try:
                        prices.append(d.find_element_by_id("priceblock_dealprice").text.strip())
                    except NoSuchElementException:
                        try:
                            prices.append(d.find_element_by_id("priceblock_saleprice").text.strip())
                        except NoSuchElementException:
                            prices.append("-")
                try:
                    expected_delivery.append(d.find_element_by_id("ddmDeliveryMessage").text.strip())
                except NoSuchElementException:
                    expected_delivery.append("-")
                try:
                    avl = d.find_element_by_id("availability").text.strip()
                    if len(avl) > 0:
                        availabilities.append(avl)
                    else:
                        availabilities.append("-")
                except NoSuchElementException:
                    availabilities.append("-")
                prod_urls.append(d.current_url)
                try:
                    other_details.append(d.find_element_by_id('feature-bullets').text)
                except NoSuchElementException:
                    other_details.append("-")
                try:
                    returns.append(d.find_element_by_xpath("//div[@id='RETURNS_POLICY']//a").text)
                except NoSuchElementException:
                    returns.append("-")
            except StaleElementReferenceException:
                d.navigate().refresh()
                time.sleep(2)
                try:
                    brands.append(d.find_element_by_id("bylineInfo").text)
                except NoSuchElementException:
                    brands.append("-")
                try:
                    product_names.append(d.find_element_by_id("productTitle").text.strip())
                except NoSuchElementException:
                    product_names.append("-")
                try:
                    rating = d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@class='a-icon-alt']").text.strip()
                    ratings.append(rating)
                    num_ratings.append(d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@id='acrCustomerReviewText']").text.strip())
                except NoSuchElementException:
                    ratings.append("-")
                    num_ratings.append("-")            
                try:
                    prices.append(d.find_element_by_id("priceblock_ourprice").text.strip())
                except NoSuchElementException:
                    try:
                        prices.append(d.find_element_by_id("priceblock_dealprice").text.strip())
                    except NoSuchElementException:
                        try:
                            prices.append(d.find_element_by_id("priceblock_saleprice").text.strip())
                        except NoSuchElementException:
                            prices.append("-")
                try:
                    expected_delivery.append(d.find_element_by_id("ddmDeliveryMessage").text.strip())
                except NoSuchElementException:
                    expected_delivery.append("-")
                try:
                    avl = d.find_element_by_id("availability").text.strip()
                    if len(avl) > 0:
                        availabilities.append(avl)
                    else:
                        availabilities.append("-")
                except NoSuchElementException:
                    availabilities.append("-")
                prod_urls.append(d.current_url)
                try:
                    other_details.append(d.find_element_by_id('feature-bullets').text)
                except NoSuchElementException:
                    other_details.append("-")
                try:
                    returns.append(d.find_element_by_xpath("//div[@id='RETURNS_POLICY']//a").text)
                except NoSuchElementException:
                    returns.append("-")

            if next_tab:
                d.close()
                d.switch_to.window(main_window)
            else:
                d.back()



        try:
            next_page_url = d.find_element_by_xpath("//li[@class='a-last']//a").get_attribute('href')
            d.quit()
            d = get_drive_launch(next_page_url)

        except NoSuchElementException:
            break
    df = pd.DataFrame({'Brand Name':brands, "Name of the Product": product_names, "Rating":ratings, "No. of Ratings": num_ratings, "Price": prices, "Expected Delivery":expected_delivery, "Availability":availabilities,"Returns/Exchange": returns, "Other Details": other_details})
    if save_as_csv:
        df.to_csv(product+"_details.csv", sep=',')
    return d, df

In [ ]:
#pd.DataFrame({'Brand Name':brands, "Name of the Product": product_names, "Rating":ratings, "No. of Ratings": num_ratings, "Price": prices, "Expected Delivery":expected_delivery, "Availability":availabilities})
d, data = fetch_product_details_amazon('guitar',3,True)
d.quit()
data

## 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.


In [23]:
d = get_drive_launch("https://images.google.com/")

d.find_element_by_xpath("//input[@title='Search']").send_keys("fruits")
d.find_element_by_xpath("//button[@type='submit']").click()

## 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [31]:
def get_long_lat_gmap_for(city):
    dr = get_drive_launch("https://www.google.com/maps")

    dr.find_element_by_id("searchboxinput").send_keys(city)
    dr.find_element_by_id("searchbox-searchbutton").click()
    try:
        element = WebDriverWait(dr, 20).until(
            EC.presence_of_element_located((By.XPATH, "//button[@data-value='Directions']"))
        )
    finally: 
        cur_url = dr.current_url

    cur_url = cur_url.split('@')[1].split(',')
    dr.quit()
    return "Longitude: "+cur_url[0],"Latitude: "+cur_url[1]

In [28]:
def get_longitude_latitude_for(city):
    url = f'https://nominatim.openstreetmap.org/search/{city}?format=json'
    response = requests.get(url).json()
    return "Longitude: "+response[0]["lat"], "Latitude: "+response[0]["lon"]

In [29]:
get_long_lat_gmap_for('new york')

('Longitude: 40.6971494', 'Latitude: -74.2598684')

In [30]:
get_longitude_latitude_for('new york')

('Longitude: 40.7127281', 'Latitude: -74.0060152')

## 6.Write a program to scrap details of all the funding deals for Third quarter (i.e. July 20 - September 20) from trak.in.

In [1]:
from selenium.webdriver.support.ui import Select

In [25]:
def get_details_trak(quarter, year):
    url = "https://trak.in/"
    q = quarter
    year = str(year)
    months = ['January','February','March','April','May','June','July','August','September','October','November','December']
    dates = []
    start_ups = []
    verticals = []
    sub_verticals =[]
    cities = []
    investors = [] 
    invest_types = []
    amts = []

    print(f"Looking for Quarter {q} details in {year}...")

    if q==1:
        months_window = months[0:3]
    elif q==2:
        months_window = months[3:6]
    elif q==3:
        months_window = months[6:9]
    elif q==4:
        months_window = months[9:]



    d = get_drive_launch(url)
    d.find_element_by_xpath("//div[@class='container']//a[contains(text(),'Funding Deals')]").click()
    time.sleep(3)
    for month in months_window:
        month = month+", "+year

        print(f'Scrapping details for the month {month}...')
        while True:
            
            try:
                date_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-2']")
                if len(date_elemns) < 1:
                    print(f'Details for {month} not found!')
                    break                 
                start_up_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-3']")
                vertical_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-4']")
                sub_vertical_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-5']")
                city_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-6']")
                investors_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-7']")
                invest_type_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-8']")
                amt_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-9']/strong")
            except NoSuchElementException:
                print(f'Details for {month} not found!')
                break

            for date_elemn,start_up_elemn,vertical_elemn,sub_vertical_elemn,city_elemn,investors_elemn,invest_type_elemn,amt_elemn in zip(date_elemns,start_up_elemns,vertical_elemns,sub_vertical_elemns,city_elemns,investors_elemns,invest_type_elemns,amt_elemns):
                dates.append(date_elemn.text)
                start_ups.append(start_up_elemn.text)
                verticals.append(vertical_elemn.text)
                sub_verticals.append(sub_vertical_elemn.text)
                cities.append(city_elemn.text)
                investors.append(investors_elemn.text)
                invest_types.append(invest_type_elemn.text)
                amts.append(amt_elemn.text)

            try:
                d.find_element_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//a[@class='paginate_button next']").click()
                time.sleep(2)
            except (NoSuchElementException,ElementClickInterceptedException):
                break
    return d, pd.DataFrame({'Date':dates, "Startup Name": start_ups, "Industry/Vertical":verticals, "Sub-Vertical":sub_verticals, "City/Location":cities, "Investors' Name":investors, "Investment type":invest_types, "Amount(In USD)":amts})

In [26]:
d, df = get_details_trak(3,2020)
d.quit()
df

Looking for Quarter 3 details in 2020...
Scrapping details for the month July, 2020...
Scrapping details for the month August, 2020...
Scrapping details for the month September, 2020...


,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investors' Name,Investment type,Amount(In USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [27]:
d, df = get_details_trak(3,2021)
d.quit()
df

Looking for Quarter 3 details in 2021...
Scrapping details for the month July, 2021...
Details for July, 2021 not found!
Scrapping details for the month August, 2021...
Details for August, 2021 not found!
Scrapping details for the month September, 2021...
Details for September, 2021 not found!


,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investors' Name,Investment type,Amount(In USD)


In [ ]:
cur_url = d.current_url
for product_elemn in range(len(search_results_elemns)):
    search_results_elemns[product_elemn].click()
    time.sleep(2)
    print(d.find_element_by_xpath("//h1").text)
    d.get(cur_url)
    time.sleep(3)
    search_results_elemns = d.find_elements_by_class_name("searchPage")

In [23]:
url = "https://www.digit.in/"
products = []

d = get_drive_launch(url)
time.sleep(3)
d.find_element_by_class_name("search").click()
time.sleep(1)
d.find_element_by_id("globalPageSearchText").send_keys("gaming laptops")
time.sleep(2)
d.find_element_by_id("globalPageSearchText").send_keys(Keys.RETURN)
time.sleep(2)
d.find_element_by_id("content_top10").click()
search_results_elemn = d.find_element_by_class_name("searchPage")
search_results_elemn.click()
print(d.find_element_by_xpath("//h1").text)


Best Gaming Laptops in India


In [16]:
d.find_element_by_id("globalPageSearchText").send_keys(Keys.RETURN)